Deadline: 23:59 on 2021 October 18

# TODO

* Prepare data
    * Images to tensors
    * Bounding box infomration to numpy array
* Implement two different models
    * Representation of the images as training input
    * Representation of the bounding boxes as objective
        * HINT: the binary classification of pixels as belonging to a bounding box or not
* Test and evaluate
    * Way 1: choose a probability threshold to decide whether a pixel is inside the bounding box or not, and then take recall/precision/X11/accuracy
    * Way 2: report it in terms of error, such as mean squared error


In [1]:
import json
import jsonlines
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from torchvision import transforms

source_dir = "/scratch/lt2326-h21/a1/"
image_dir = "/scratch/lt2326-h21/a1/images/"
info_filename = "/scratch/lt2326-h21/a1/info.json"

In [2]:
image_files = [f for f in listdir(image_dir) if isfile(join(image_dir, f))]

In [3]:
with open(info_filename, 'r') as info_file:
    data = info_file.read()
info_obj = json.loads(data)

In [4]:
df_train = pd.DataFrame.from_dict(info_obj['train'])
df_val = pd.DataFrame.from_dict(info_obj['val'])
df = pd.concat([df_train, df_val])

In [5]:
df = df[pd.DataFrame(df.file_name.to_list()).isin(image_files).any(1).values]

In [6]:
bb_data = []

with jsonlines.open(source_dir + "train.jsonl") as reader:
   for line in reader.iter(type=dict):
      if (line.get('file_name') in image_files):
         bb_data.append(line)

with jsonlines.open(source_dir + "val.jsonl") as reader:
   for line in reader.iter(type=dict):
      if (line.get('file_name') in image_files):
         bb_data.append(line)

In [8]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#testimg = mpimg.imread(image_dir + "0000486.jpg")
#imgarray= np.array(testimg)
convert_tensor = transforms.ToTensor()
# convert_tensor(imgarray)

In [ ]:
training_data = [] # lista av tuples, med bild och ground truth

for entry in bb_data:
    # convert image file to tensor
    # img = mpimg.imread(image_dir + entry['file_name'])
    # convert_tensor(img)
    # create 2048 x 2048 numpy array
    # add bounding boxes to the new array
    # add img tensor and bounding box array as tuples to training_data
    
    pass
